In [16]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from utility import p4
from utility.p5 import *
import pickle 
import time
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
from scipy.ndimage.measurements import label
from multiprocessing import Pool, cpu_count

pool = Pool(processes=cpu_count())
print("cpu_count: ", cpu_count())
%matplotlib inline

cpu_count:  4


In [17]:
f = open('camera_cals.p', 'rb')   
camera_cal = pickle.load(f) 
objpoints = camera_cal['objpoints']
imgpoints = camera_cal['imgpoints']

f = open('perspective_transformation.p', 'rb')
perspective_transformation = pickle.load(f) 

M = perspective_transformation['M']
Minv = perspective_transformation['Minv']

In [18]:
# Divide up into cars and notcars
notcar_images = glob.glob('./data/non-vehicles/**/*.png')
car_images = glob.glob('./data/vehicles/**/*.png')
cars = []
notcars = []
for image in notcar_images:
    notcars.append(image)
for image in car_images:
    cars.append(image)
    
print('car:', len(cars), 'not car:', len(notcars))

car: 8792 not car: 8968


In [19]:
car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

Using: 9 orientations 16 pixels per cell and 2 cells per block
Feature vector length: 1788


In [20]:
### TODO: Tweak these parameters and see how the results change.
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 16 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
x_start_stop = [None, None]

y_start_stop_L = [400, 656] # Min and max in y to search in slide_window()
y_start_stop_N = [400, 528] # Min and max in y to search in slide_window()



In [21]:
# Use a linear SVC 
#svc = LinearSVC()
svc = SVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

print(round(t-t2, 2), 'Seconds to predict.')


f = open('svc_linear.p', 'wb')   
pickle.dump(svc, f)

68.15 Seconds to train SVC...
Test Accuracy of SVC =  0.9969
13.86 Seconds to predict.


In [22]:
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows
    
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (255,0,0), 3)
    # Return the image
    return img

In [23]:
def detect_car(image):
    draw_image = np.copy(image)
    heat = np.zeros_like(image[:,:,0]).astype(np.float)

    windows_L = slide_window(draw_image, x_start_stop=x_start_stop, y_start_stop=y_start_stop_L, 
                        xy_window=(128, 128), xy_overlap=(0.7, 0.7))
    windows_N = slide_window(draw_image, x_start_stop=x_start_stop, y_start_stop=y_start_stop_N, 
                        xy_window=(64, 64), xy_overlap=(0.7, 0.7))
#    windows_S = slide_window(draw_image, x_start_stop=x_start_stop, y_start_stop=y_start_stop_S, 
#                        xy_window=(32, 32), xy_overlap=(0.7, 0.7))

    windows = np.concatenate((windows_L, windows_N), axis=0)
    print(len(windows))
    hot_windows = search_windows(draw_image, windows, svc, X_scaler, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)                       
    
#    for threads in cpu_count():
        
#    async_result = pool.apply_async(search_windows, ('world', 'foo')) # tuple of args for foo

    # Add heat to each box in box list
    heat = add_heat(heat,hot_windows)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,2)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    return draw_img, heatmap

In [24]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

fontFace = cv2.FONT_HERSHEY_SIMPLEX
fontScale=1
thickness=2
    
def process_image(image):
    undistorted = p4.cal_undistort(image, objpoints, imgpoints)
    combined = p4.combinedThresholds(undistorted)
    
    binary_warped = p4.warper(combined, M)
#    closing = cv2.morphologyEx(binary_warped, cv2.MORPH_CLOSE, kernel)
    global left_fitx
    global right_fitx
    left_fitx, right_fitx, left_curverad, right_curverad, out_img, left_fit, right_fit = p4.fitlines(binary_warped, left_fitx, right_fitx,lf_history, rf_history)

    lf.append(left_fit)
    rf.append(right_fit)
    
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = p4.warper(color_warp, Minv) 
    # Combine the result with the original image
    result = cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)
    
    result,heatmap = detect_car(result.astype(np.float32) /255  )
    
    result = (result*255).astype(np.uint8)
    heatmap = (heatmap*255).astype(np.uint8)
    
    middle = int((left_fitx[-1] + right_fitx[-1])//2)
    veh_pos = int(image.shape[1]//2)
    
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    dx = (veh_pos - middle)*xm_per_pix 
    
    #draw position lines    
    cv2.line(result, (veh_pos, 400), (veh_pos, 720), (200,200,200),2)
    cv2.line(result, (veh_pos-20, 400), (veh_pos+20, 400), (200,200,200),2)
    cv2.line(result, (veh_pos-30, 450), (veh_pos+30, 450), (200,200,200),2)
    cv2.line(result, (veh_pos-20, 500), (veh_pos+20, 500), (200,200,200),2)
    cv2.line(result, (middle, 550), (middle, 720), (200,200,200),4)

    img_out=np.zeros((576,1280,3), dtype=np.uint8)
    img_out[0:576,0:1024,:] =cv2.resize(result,(1024,576))
    #b) Threshold
    img_out[0:192,1024:1280, 0] =cv2.resize(combined,(256,192))
    img_out[0:192,1024:1280, 1] =cv2.resize(combined,(256,192))
    img_out[0:192,1024:1280, 2] =cv2.resize(combined,(256,192))
    #c)Birds eye view
    img_out[192:384,1024:1280,:] =cv2.resize(out_img,(256,192))
    img_out[384:576,1024:1280,0] =cv2.resize(heatmap,(256,192))
    img_out[384:576,1024:1280,1] =cv2.resize(heatmap,(256,192))
    img_out[384:576,1024:1280,2] =cv2.resize(heatmap,(256,192))
#    global path
#    global counter
#    cv2.imwrite('images/{}/{}.jpg'.format(path, counter), image)
#    counter += 1
    
    #Write curvature and center in image
    TextL = "Left  curv: {:>.1f} km".format(left_curverad/1000)
    TextR = "Right curv: {:>.1f} km".format(right_curverad/1000)

    cv2.putText(img_out, TextL, (130,40), fontFace, fontScale,(200,200,200), thickness,  lineType = cv2.LINE_AA)
    cv2.putText(img_out, TextR, (130,70), fontFace, fontScale,(200,200,200), thickness,  lineType = cv2.LINE_AA)
    cv2.putText(img_out,'Position : %.2f m %s of center'%(abs(dx), 'left' if dx < 0 else 'right'),(50,110), 
                        fontFace, fontScale,(200,200,200),thickness,lineType = cv2.LINE_AA)

    cv2.putText(img_out, "Thresh. view", (1070,20), fontFace, .8,(200,200,0), thickness,  lineType = cv2.LINE_AA)
    cv2.putText(img_out, "Birds-eye view", (1070,212), fontFace, .8,(200,200,0), thickness,  lineType = cv2.LINE_AA)
    cv2.putText(img_out, "Heat map", (1070,404), fontFace, .8,(200,200,0), thickness,  lineType = cv2.LINE_AA)
    
    return img_out

In [25]:
path='project'
counter = 1
lf = []
rf = []
left_fitx= [0,0,0]
right_fitx = [0,0,0]
lf_history = np.zeros((5,3))
rf_history = np.zeros((5,3))
white_output = 'project_video_processe.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image).subclip(38,42) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

26312  -  12086
384
42111  -  8361
384
[MoviePy] >>>> Building video project_video_processe.mp4
[MoviePy] Writing video project_video_processe.mp4


  0%|          | 0/101 [00:00<?, ?it/s]

42111  -  8361
384


  1%|          | 1/101 [00:03<05:11,  3.11s/it]

42386  -  8632
384


...

38677  -  7447
384


 98%|█████████▊| 99/101 [05:09<00:06,  3.15s/it]

38443  -  7402
384


 99%|█████████▉| 100/101 [05:12<00:03,  3.15s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_processe.mp4 

CPU times: user 5min 20s, sys: 170 ms, total: 5min 20s
Wall time: 5min 13s
